<a href="https://colab.research.google.com/github/linhvien/Stock-analysis/blob/main/Vietnamese_stock_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import libraries

In [9]:
from bs4 import BeautifulSoup as bs
import requests

import re
import json
import csv
from io import StringIO
from datetime import datetime, timedelta
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Overview

In [35]:
start_date = '2019-01-01'
end_date = str(datetime.now().strftime('%Y-%m-%d'))
stock = 'VIC'
portfolio = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

#Financial data
Loading quarterly financial data of Vietnamese stocks from api vndirect.

In [11]:
class Financialdata:
  url_BS = 'https://finfo-api.vndirect.com.vn/v3/stocks/financialStatement?secCodes={}&reportTypes=QUARTER&modelTypes=1,89,101,411&fromDate={}&toDate={}'
  url_IS = 'https://finfo-api.vndirect.com.vn/v3/stocks/financialStatement?secCodes={}&reportTypes=QUARTER&modelTypes=2,90,102,412&fromDate={}&toDate={}'
  url_CF = 'https://finfo-api.vndirect.com.vn/v3/stocks/financialStatement?secCodes={}&reportTypes=QUARTER&modelTypes=3,91,103,413&fromDate={}&toDate={}'
  url_ratio = 'https://finfo-api.vndirect.com.vn/v4/ratios?q=code:{}~itemCode:53030,52005,51050,53021,52001,52002,54018,712010,712020,712030,712040~reportDate:{}'
  
  def __init__(self,stock,start_date,end_date,*arg, **karg):
    self.stock = stock
    self.start_date = start_date
    self.end_date = end_date
    self.session = requests.Session()

# Get balance sheet
  def get_BS(self):
    try:
      page = self.session.get(self.url_BS.format(self.stock,self.start_date,self.end_date), headers=headers)
      data = page.json()

      data_dates = {}
      for item in data['data']['hits']:
        item = item['_source']
        date = item['fiscalDate']
        itemName = item['itemName']
        numericValue = item['numericValue']
        if date not in data_dates:
          data_dates[date] = [[], []]
        else:
          if itemName not in data_dates[date][0]:
            data_dates[date][0].append(itemName)
            data_dates[date][1].append(numericValue)

      for i, (date, item) in enumerate(data_dates.items()):
        df_date = pd.DataFrame(data={'index':item[0], date[:7]:item[1]})
        if i == 0:
          df = df_date
        else:
          df = pd.merge(df, df_date, how='inner')
      df.set_index('index', inplace=True)
    except:
      pass
    return df

# get income statement
  def get_IS(self):
    try:
      page = self.session.get(self.url_IS.format(self.stock,self.start_date,self.end_date), headers=headers)
      data = page.json()

      data_dates = {}
      for item in data['data']['hits']:
        item = item['_source']
        date = item['fiscalDate']
        itemName = item['itemName']
        numericValue = item['numericValue']
        if date not in data_dates:
          data_dates[date] = [[], []]
        else:
          if itemName not in data_dates[date][0]:
            data_dates[date][0].append(itemName)
            data_dates[date][1].append(numericValue)

      for i, (date, item) in enumerate(data_dates.items()):
        df_date = pd.DataFrame(data={'index':item[0], date[:7]:item[1]})
        if i == 0:
          df = df_date
        else:
          df = pd.merge(df, df_date, how='inner')
      df.set_index('index', inplace=True)
    except:
      pass
    return df

# get cashflow statement
  def get_CF(self):
    try:
      page = self.session.get(self.url_CF.format(self.stock,self.start_date,self.end_date), headers=headers)
      data = page.json()

      data_dates = {}
      for item in data['data']['hits']:
        item = item['_source']
        date = item['fiscalDate']
        itemName = item['itemName']
        numericValue = item['numericValue']
        if date not in data_dates:
          data_dates[date] = [[], []]
        else:
          if itemName not in data_dates[date][0]:
            data_dates[date][0].append(itemName)
            data_dates[date][1].append(numericValue)

      for i, (date, item) in enumerate(data_dates.items()):
        df_date = pd.DataFrame(data={'index':item[0], date[:7]:item[1]})
        if i == 0:
          df = df_date
        else:
          df = pd.merge(df, df_date, how='inner')
      df.set_index('index', inplace=True)
    except:
      pass
    return df

In [12]:
BS_data = Financialdata(stock,start_date,end_date).get_BS()
BS_data.to_excel(r'BS_{}.xlsx'.format(stock),index=True)
BS_data.head()

,2021-06,2021-03,2020-12,2020-09,2020-06,2020-03,2019-12,2019-09,2019-06,2019-03
index,,,,,,,,,,
Tiền và các khoản tương đương tiền,1663036014269,1197956795582,2111242815581,2335881481579,2176667357252,1379711866875,2665194638452,1043473767363,1676491290318,1209154406312
Tiền,1261476873446,869981048166,863853260384,995896481579,2044667357252,1028411866875,2378583764655,987873767363,826491290318,1029154406312
Các khoản tương đương tiền,401559140823,327975747416,1247389555197,1339985000000,132000000000,351300000000,286610873797,55600000000,850000000000,180000000000
Các khoản đầu tư tài chính ngắn hạn,19971222710510,19542794237097,17313679774893,17872002664622,16363060738340,14370288608530,12435744328964,10238285848104,9306531479431,9666846652579
Đầu tư ngắn hạn,1129553032,1123201739,1124178861,1128087349,1129064471,1148118350,1153041048,443154506731,443157926658,443154995292


In [13]:
IS_data = Financialdata(stock,start_date,end_date).get_IS()
IS_data.to_excel(r'IS_{}.xlsx'.format(stock),index=True)
IS_data.head()

,2021-06,2021-03,2020-12,2020-09,2020-06,2020-03,2019-12,2019-09,2019-06,2019-03
index,,,,,,,,,,
Các khoản giảm trừ doanh thu,12980283496,50444960886,5105999997,14071686928,14352408276,53092072488,16738916476,13142547924,11441072483,40784427090
Doanh thu thuần,15715806166465,13190270122852,14424794514944,15563157998229,15495233758799,14153099953575,14238961478162,14290900447018,14598960549757,13189300287807
Giá vốn hàng bán,8861448981220,7435389686438,7766620813396,8296414517093,8357339161700,7547288345650,7558658856714,7567934476550,7585015000599,7034297778254
Lợi nhuận gộp,6854357185245,5754880436414,6658173701548,7266743481136,7137894597099,6605811607925,6680302621448,6722965970468,7013945549158,6155002509553
Doanh thu hoạt động tài chính,292767071141,284839944505,602270330836,358591147428,335351494778,284879682275,233576605377,206732692690,190720049829,176287359587


In [14]:
CF_data = Financialdata(stock,start_date,end_date).get_CF()
CF_data.to_excel(r'CF_{}.xlsx'.format(stock),index=True)
CF_data.head()

,2021-06,2021-03,2020-12,2020-09,2020-06,2020-03,2019-12,2019-09,2019-06,2019-03
index,,,,,,,,,,
Chi phí khấu hao tài sản cố định,547756789868,499689193833,545413737797,551231227788,556761193198,555460462969,502928809106,494141498125,478654208826,472348259477
Phân bổ lợi thế thương mại,61385028806,61385028806,349960385329,123278040202,67545449401,67364700041,16987979595,16987979595,16987979595,16987979595
"Dự phòng giảm giá các khoản đầu tư ngắn hạn, dài hạn",-3508982145,3789250096,43717904388,-2000594719,6782636573,1003914771,1003332731,-748428436,2076193560,5001735507
"Lãi, lỗ chênh lệch tỷ giá hối đoái chưa thực hiện",-3585620065,-1281139009,17776990028,-14810127185,-70182197587,67852556191,7871927410,-678177168,468890998,-1204432181
Lãi/(lỗ) từ thanh lý tài sản cố định,16223729056,5187526637,14082316639,9964216884,10375293005,8717274881,6348242528,13281469680,10580680034,13753677997


#Full stock and industry

In [18]:
# Get industry list from vietstock
r = requests.get('https://finance.vietstock.vn/chi-so-nganh.htm',headers=headers)
soup = bs(r.content)
def Getsectorname():
  sectors = soup.select('a[href^="/nganh"]')
  all_sector = []
  for i in sectors:
    sector_name = i.get('title')
    if sector_name not in all_sector:
      all_sector.append(sector_name)
  return all_sector

In [19]:
Getsectorname()

['Bán buôn',
 'Bảo hiểm',
 'Bất động sản',
 'Chứng khoán',
 'Công nghệ và thông tin',
 'Bán lẻ',
 'Chăm sóc sức khỏe',
 'Khai khoáng',
 'Ngân hàng',
 'Nông - Lâm - Ngư',
 'SX Thiết bị, máy móc',
 'SX Hàng gia dụng',
 'Sản phẩm cao su',
 'SX Nhựa - Hóa chất',
 'Thực phẩm - Đồ uống',
 'Chế biến Thủy sản',
 'Vật liệu xây dựng',
 'Tiện ích',
 'Vận tải - kho bãi',
 'Xây dựng',
 'Dịch vụ lưu trú, ăn uống, giải trí',
 'SX Phụ trợ',
 'Thiết bị điện',
 'Dịch vụ tư vấn, hỗ trợ',
 'Tài chính khác']

In [20]:
def GetURL():
    sectors = soup.select('a[href^="/nganh"]')
    all_sector_URL = []
    for sector in sectors:
        sector_ID = sector.get('href')
        sector_URL = 'https://finance.vietstock.vn' + sector_ID
        if sector_URL not in all_sector_URL:
            all_sector_URL.append(sector_URL)
    return all_sector_URL

In [21]:
url = GetURL()
url

['https://finance.vietstock.vn/nganh/1-ban-buon.htm',
 'https://finance.vietstock.vn/nganh/2-bao-hiem.htm',
 'https://finance.vietstock.vn/nganh/3-bat-dong-san.htm',
 'https://finance.vietstock.vn/nganh/5-chung-khoan.htm',
 'https://finance.vietstock.vn/nganh/6-cong-nghe-va-thong-tin.htm',
 'https://finance.vietstock.vn/nganh/7-ban-le.htm',
 'https://finance.vietstock.vn/nganh/8-cham-soc-suc-khoe.htm',
 'https://finance.vietstock.vn/nganh/10-khai-khoang.htm',
 'https://finance.vietstock.vn/nganh/11-ngan-hang.htm',
 'https://finance.vietstock.vn/nganh/12-nong-lam-ngu.htm',
 'https://finance.vietstock.vn/nganh/15-sx-thiet-bi-may-moc.htm',
 'https://finance.vietstock.vn/nganh/16-sx-hang-gia-dung.htm',
 'https://finance.vietstock.vn/nganh/17-san-pham-cao-su.htm',
 'https://finance.vietstock.vn/nganh/18-sx-nhua-hoa-chat.htm',
 'https://finance.vietstock.vn/nganh/19-thuc-pham-do-uong.htm',
 'https://finance.vietstock.vn/nganh/20-che-bien-thuy-san.htm',
 'https://finance.vietstock.vn/nganh/21

In [22]:
#Get full stocks along with industry
full_stock =[]
for i in url:
  industry = pd.read_html(requests.get(i,headers=headers).text)
  industry = pd.DataFrame(industry[1])
  full_stock.append(industry)
symbols = pd.concat(full_stock, axis=0, ignore_index=True)
symbols = symbols.drop(columns=['STT','Sàn','Giá 1 ngày'])
symbols.tail()

,Mã CK,Ngành cấp 3,KLCPLH,Giá 5 ngày
735,VCM,Dịch vụ việc làm,3000000,15200
736,VLA,Dịch vụ lập trình máy tính,1080000,17000
737,VNC,"Các dịch vụ chuyên môn, khoa học và kỹ thuật khác",10499560,33000
738,OGC,Các hoạt động đầu tư tài chính khác,299999999,7270
739,TVC,Các hoạt động đầu tư tài chính khác,103610670,18000


#Real-time price
Get real time price for stock

In [ ]:
def getrealtimeprice(stock):
  r = requests.get('https://finance.vietstock.vn/{}/thong-ke-giao-dich.htm'.format(stock),headers=headers)
  soup = bs(r.content)
  price = soup.select('h2#stockprice span')[0].text
  price = price.replace(',', '')
  return int(price)

#Financial ratio analysis
Create function to calculate financial ratio for stock list

In [75]:
stocklist=[] 
def FinRatio(stocklist):
  FinRatio = {}
  for company in stocklist:
    BS = Financialdata(company,start_date,end_date).get_BS()
    IS = Financialdata(company,start_date,end_date).get_IS()
    CF = Financialdata(company,start_date,end_date).get_CF()
    price = getrealtimeprice(company)
    KLCPLH = int(symbols.loc[symbols['Mã CK'] == company, 'KLCPLH'].to_string(index=False).strip())
    market_cap = KLCPLH * price

    FinRatio[company] = {}
    FinRatio[company]['market_cap'] = market_cap
    FinRatio[company]['Trailing PE'] = market_cap/(IS.loc['Lợi nhuận sau thuế thu nhập doanh nghiệp'][0:4].sum())
    FinRatio[company]['ROE'] = IS.loc['Doanh thu thuần'].mean()/BS.loc['Vốn chủ sở hữu'].mean()
    FinRatio[company]['ROA'] = IS.loc['Doanh thu thuần'].mean()/BS.loc['TỔNG CỘNG TÀI SẢN'].mean()
    FinRatio[company]['EPS 4 latest quart'] = IS.loc['Lãi cơ bản trên cổ phiếu'][0:4].sum()
    FinRatio[company]['Operating margin'] = IS.loc['Lợi nhuận thuần từ hoạt động kinh doanh'].mean()/IS.loc['Doanh thu thuần'].mean()
    

  df = pd.DataFrame.from_dict(FinRatio,orient='index')
  return df

##Portfolio analysis

In [76]:
portfolio = ['VNM','VIC']
FinRatio(portfolio)

,market_cap,Trailing PE,ROE,ROA,EPS 4 latest quart,Operating margin
VNM,185588043516000,17.130841,0.461767,0.313625,4572,0.227239
VIC,327628993505100,73.529054,0.251776,0.080582,476,0.116875


##Peers analysis

In [77]:
# Get stock list in the same sector with selected stock
industry = symbols.loc[symbols['Mã CK'] == stock, 'Ngành cấp 3'].to_string(index=False).strip()
stocklist = symbols['Mã CK'].loc[symbols['Ngành cấp 3'] == industry].tolist()

# Compare ratios between stocks in the same sector
FinRatio(stocklist)

,market_cap,Trailing PE,ROE,ROA,EPS 4 latest quart,Operating margin
AGG,3533449637900,7.879522,0.154683,0.038605,5093,0.365914
API,1451400000000,25.325382,0.256949,0.051637,1618,0.121877
BCE,463750000000,9.024393,0.318050,0.120149,1390,0.089716
CDC,322134689400,17.288353,0.378840,0.118297,917,0.076511
CEO,2496197854500,-15.791325,0.187452,0.078589,0,0.115472
CIG,237180401440,-8.384013,0.012321,0.003753,0,-6.401177
CKG,1926369886350,13.654407,0.398821,0.063262,2558,0.141595
D11,165109518000,5.082247,0.581124,0.104398,3960,0.133229
D2D,1549298790400,6.840233,0.160793,0.062652,0,0.738141
DIG,12491794282800,18.382997,0.139612,0.063821,0,0.165603


# Historical data

In [25]:
def convert_date(text, date_type = '%Y-%m-%d'):
  return datetime.strptime(text, date_type)
  
def convert_text_dateformat(text, origin_type = '%Y-%m-%d', new_type = '%Y-%m-%d'):
  return convert_date(text, origin_type).strftime(new_type)

def split_change_col(text):
    return re.sub(r'[\(|\)%]', '', text).strip().split()

class DataLoadProto():
    def __init__(self, symbols, start, end, *arg, **karg):
        self.symbols = symbols
        self.start = convert_text_dateformat(start, new_type = '%d/%m/%Y')
        self.end = convert_text_dateformat(end, new_type = '%d/%m/%Y')

class DataLoaderCAFE(DataLoadProto):
    def __init__(self, symbols, start, end, *arg, **karg):
        self.symbols = symbols
        self.start = start
        self.end = end
        super(DataLoaderCAFE, self).__init__(symbols, start, end)

    def download(self):
        stock_datas = []
        if not isinstance(self.symbols, list):
            symbols = [self.symbols]
        else:
            symbols = self.symbols

        for symbol in symbols:
            stock_datas.append(self.download_one(symbol))

        data = pd.concat(stock_datas, axis=1)
        return data

    def download_one(self, symbol):
        stock_data = pd.DataFrame(columns=['date', 'change_perc1', 'change_perc2',
                                           'open', 'high', 'low', 'close',
                                           'avg', 'volume_match', 'volume_reconcile'])

        for i in range(1000):
            stock_slice_batch = self.download_batch(i + 1, symbol)
            stock_data = pd.concat([stock_data, stock_slice_batch], axis=0)
            try:
                date_end_batch = stock_slice_batch.date.values[-1]
            except:
                # start date is holiday or weekend
                break
            is_touch_end = convert_date(self.start, '%d/%m/%Y') == convert_date(date_end_batch, '%d/%m/%Y')
            # logging.info('batch: {}; start date out range: {}; date_end_batch: {}'.format(i + 1, is_touch_end, date_end_batch))
            if is_touch_end:
                break

        stock_data['change_perc1'], stock_data['change_perc2'] = stock_data['change_perc'].apply(split_change_col).str
        if 'change_perc' in stock_data.columns:
            stock_data.pop('change_perc')
        if 'avg' in stock_data.columns:
            stock_data.pop('avg')
            stock_data = stock_data.set_index('date').apply(pd.to_numeric, errors='coerce')
            stock_data.index = list(map(lambda text: convert_date(text, date_type='%d/%m/%Y'), stock_data.index))
            stock_data.index.name = 'date'
            stock_data = stock_data.sort_index()
            stock_data.fillna(0, inplace=True)
            stock_data['volume'] = stock_data.volume_match + stock_data.volume_reconcile

        # Create multiple columns
        iterables = [stock_data.columns.tolist(), [symbol]]
        mulindex = pd.MultiIndex.from_product(iterables, names=['Attributes', 'Symbols'])
        stock_data.columns = mulindex


        logging.info('data {} from {} to {} have already cloned!' \
                     .format(symbol,
                             convert_text_dateformat(self.start, origin_type = '%d/%m/%Y', new_type = '%Y-%m-%d'),
                             convert_text_dateformat(self.end, origin_type='%d/%m/%Y', new_type='%Y-%m-%d')))

        return stock_data

    def download_batch(self, id_batch, symbol):
        form_data = {'ctl00$ContentPlaceHolder1$scriptmanager':'ctl00$ContentPlaceHolder1$ctl03$panelAjax|ctl00$ContentPlaceHolder1$ctl03$pager2',
                       'ctl00$ContentPlaceHolder1$ctl03$txtKeyword':symbol,
                       'ctl00$ContentPlaceHolder1$ctl03$dpkTradeDate1$txtDatePicker':self.start,
                       'ctl00$ContentPlaceHolder1$ctl03$dpkTradeDate2$txtDatePicker':self.end,
                       '__EVENTTARGET':'ctl00$ContentPlaceHolder1$ctl03$pager2',
                       '__EVENTARGUMENT':id_batch,
                       '__ASYNCPOST':'true'}
        url = URL_CAFE+symbol+"-1.chn"
        r = requests.post(url, data = form_data, headers = headers, verify=False)
        soup = BeautifulSoup(r.content, 'html.parser')
        print(id_batch)
        table = soup.find('table')
        stock_slice_batch = pd.read_html(str(table))[0].iloc[2:, :12]

        stock_slice_batch.columns = ['date', 'adjust', 'close', 'change_perc', 'avg',
                        'volume_match', 'value_match', 'volume_reconcile', 'value_reconcile',
                        'open', 'high', 'low']

        return stock_slice_batch
